In [26]:
import pandas as pd
import numpy as np

In [28]:
data = pd.read_csv('/root/autodl-tmp/wq/RS_Keras/data/Sohu2022_data/rec_data/train-dataset.csv')

In [4]:
def split_item(one_data):
    '''
    将userSeq列中item添加到新列表中作为特征列
    '''
    tmp_item_list = []
    for one_item in str(one_data).split(";"):  
        tmp_item_list.append(one_item.split(':')[0])
    return tmp_item_list

In [5]:
def split_time(one_data):
    '''
    将userSeq列中time添加到新列表中作为特征列
    '''
    tmp_time_list = []
    for one_item in str(one_data).split(";"):  
        tmp_time_list.append(one_item.split(':')[-1])
    return tmp_time_list  

In [9]:
data['item_list'] = data.userSeq.apply(split_item)

In [10]:
data['time_list'] = data.userSeq.apply(split_time)

#### 准备word2vec的输入：二维列表

In [11]:
all_item_list = []
for item_list in data['item_list']:
    all_item_list.append(item_list)

In [14]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

train_model = Word2Vec(all_item_list, vector_size=50, window=5, min_count=1, workers=4)
train_model.save('./MyModel')

train_model.wv.save_word2vec_format('./item.txt', binary=False)


##### 将Wordvec训练好的embedding向量用csv格式储存

In [51]:
data_txt = np.loadtxt('./item.txt')
data_txtDF = pd.DataFrame(data_txt,columns= ['itemId']+[ f'Embedding_{i}' for i in range(50)])
data_txtDF.to_csv('item_embedding_csv',index=False)

In [29]:
item_embedding = pd.read_csv('./item_embedding_csv')

In [4]:
item_embedding.head()

,itemId,Embedding_0,Embedding_1,Embedding_2,Embedding_3,Embedding_4,Embedding_5,Embedding_6,Embedding_7,Embedding_8,...,Embedding_40,Embedding_41,Embedding_42,Embedding_43,Embedding_44,Embedding_45,Embedding_46,Embedding_47,Embedding_48,Embedding_49
0,NaN,-0.001072,0.000473,0.010207,0.018019,-0.018606,-0.014234,0.012918,0.017946,-0.010031,...,-0.019207,0.010015,-0.017519,-0.008784,-0.000070,-0.000592,-0.015322,0.019229,0.009964,0.018466
1,10099554.0,-0.287784,-4.716245,0.210488,-5.690922,-3.261221,2.821887,-6.503364,-0.596449,-1.991397,...,2.355053,4.622287,2.714019,0.301596,-0.374871,7.197029,0.149656,3.067329,-1.244174,9.874837
2,10085107.0,-5.990616,4.486032,1.748040,0.592468,3.132309,-1.635398,3.468003,3.767014,-3.580695,...,-5.682347,-5.869978,0.420320,-2.483540,1.964049,-4.782959,0.919976,-3.146383,0.455233,-8.672552
3,10089363.0,-0.059581,2.360711,-4.737878,2.268620,4.410136,4.738972,4.729690,2.870426,-1.788495,...,-9.694828,-6.637001,1.539428,-4.868769,-1.983912,-6.314855,4.631748,-2.638040,-1.109178,-7.818279
4,10097699.0,-0.152875,-1.138640,-2.968007,4.007520,9.905430,2.783760,4.534660,0.735650,0.804311,...,-1.125425,-11.210804,-4.569268,-1.992595,-1.283429,-8.627345,1.456349,-3.329998,2.244988,-2.023987


#### fasis中向量库大小为（id个数，向量维度）

In [30]:
item_embedding_array_list = []
for x in item_embedding.values:
    item_embedding_array_list.append(x[1:])
item_embedding_array_list = np.array(item_embedding_array_list).astype(np.float32)


In [31]:
import faiss
dimension = 50
#建立索引
index = faiss.IndexFlatL2(dimension)
#将向量加进faiss库中
index.add(item_embedding_array_list)

In [33]:
top_k = 5

#D为距离，I为索引
D,I = index.search(item_embedding_array_list[1:5],top_k)